## 파이썬 웹 크롤링(1) 뉴스 기사 스크랩

라이브러리

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import re

<br>

### 검색 결과에서  뉴스 제목 추려내기

네이버 뉴스탭에서 검색

In [5]:
query = '배우 박정민'
url = "https://search.naver.com/search.naver?where=news&query=" + query

web = requests.get(url).content # get 요청
source = BeautifulSoup(web, 'html.parser')

url

'https://search.naver.com/search.naver?where=news&query=배우 박정민'

![news_list](image/newslist.png)

뉴스 제목 리스트

In [6]:
subjects = source.find_all('a', {'class': 'news_tit'})

subjects_list = []

for subject in subjects:
    subjects_list.append(subject.get_text())
    
subjects_list

['배우 이제훈, 매니지먼트 컴퍼니온 설립 [공식]',
 '배우 이제훈 매니지먼트 사업 뛰어든다',
 "김혜수·조인성·염정아·박정민… 류승완 감독 신작 '밀수' 크랭크인(공식)",
 '박정민, 책방 운영 종료…"언젠가 더 번듯하게"',
 '김성철 “어릴 때부터 흥부자, 배우 꿈꿔왔다” [화보]',
 '박정민, 책방 폐업 결정 "운영에 어려운 문제 있었다"',
 '배우 박정민, 2년 만에 책방 폐점',
 "'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연",
 "류승완 감독 '밀수' 김혜수·염정아→조인성·박정민 캐스팅 확정…크랭크인",
 '박정민 "연기 위해 고려대 자퇴…후회는 자주 했다" 솔직 고백']

뉴스 url 리스트

In [8]:
for subject in subjects:
    print(subject.attrs['href'])

http://starin.edaily.co.kr/news/newspath.asp?newsid=01151286629084016
https://view.asiae.co.kr/article/2021062109425167742
https://star.ytn.co.kr/_sn/0117_202106110937275511
http://yna.kr/AKR20210607103700005?did=1195m
https://sports.donga.com/article/all/20210618/107500147/1
https://www.news1.kr/articles/?4331691
http://www.gukjenews.com/news/articleView.html?idxno=2243478
http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600
https://www.xportsnews.com/?ac=article_view&entry_id=1435543
https://www.news1.kr/articles/?4326866


In [10]:
urls_list = []
for url in subjects:
    urls_list.append(url.attrs['href']) # url['href']
    
urls_list

['http://starin.edaily.co.kr/news/newspath.asp?newsid=01151286629084016',
 'https://view.asiae.co.kr/article/2021062109425167742',
 'https://star.ytn.co.kr/_sn/0117_202106110937275511',
 'http://yna.kr/AKR20210607103700005?did=1195m',
 'https://sports.donga.com/article/all/20210618/107500147/1',
 'https://www.news1.kr/articles/?4331691',
 'http://www.gukjenews.com/news/articleView.html?idxno=2243478',
 'http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600',
 'https://www.xportsnews.com/?ac=article_view&entry_id=1435543',
 'https://www.news1.kr/articles/?4326866']

<br>

#### 단일 뉴스 내용 스크랩핑

In [13]:
urls_list[7] # 뉴시스 사 기사 기준

'http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600'

In [15]:
news_content = requests.get(urls_list[7]).content
source = BeautifulSoup(news_content, 'html.parser')

In [17]:
# 네이버에 접속이 차단당했을 때

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
news_content = requests.get(urls_list[7], headers=headers).content
source = BeautifulSoup(news_content, 'html.parser')

- 뉴스 제목

In [19]:
source.find('h1').get_text()

"'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연"

- 뉴스 발행시간

In [24]:
date_source = source.find('div', {'class': 'date'}).get_text()
date_source

'등록 2021-06-02 13:51:26'

In [31]:
year = date_source[3:7]
month = date_source[8:10]
day = date_source[11:13]

hh = date_source[14:16]
mm = date_source[17:19]
ss = date_source[20:22]
print('{}년 {}월 {}일 {}시 {}분 {}초 발행'.format(year, month, day, hh, mm, ss))

2021년 06월 02일 13시 51분 26초 발행


In [33]:
press_date = year + month + day + hh + mm + ss
press_date_ts = pd.Timestamp(press_date)
press_date_ts

Timestamp('2021-06-02 13:51:26')

- 기사본문

In [88]:
article = source.find('div', {'id': 'textBody'}).get_text()
article

'\n\n[서울=뉴시스]2일 방송되는 tvN \'유 퀴즈 온 더 블럭\'. (사진=tvN \'유 퀴즈 온 더 블럭\' 제공) 2021.06.02. photo@newsis.com[서울=뉴시스] 강진아 기자 = tvN \'유 퀴즈 온 더 블럭\'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동생 윤여순 등이 출연한다.\n\r\n2일 오후 8시40분에 방송되는 \'유 퀴즈 온 더 블럭\' 109회에서는 포기를 모르는 끝판왕 자기님들과 \'끝날 때까지 끝난 게 아니다\' 특집을 선보인다.\n\r\n배우 박정민, 대기업 첫 여성 임원 윤여순, 의사 신승건, \'플랭크\' 대가 김영달 자기님이 출연해 끝없이 경주하는 인생 이야기를 나눈다.\n\r\n먼저 장르 불문 독보적인 연기력으로 충무로 캐스팅 0순위에 오른 배우 박정민 자기님이 \'유 퀴즈\'를 찾는다. \n\r\n오로지 연기에 대한 열정으로 명문대 대신 한예종을 선택한 사연부터 전설로 남은 한예종 면접 비하인드 스토리, 무명시절을 거쳐 지금에 이르기까지 영화보다 더 영화 같은 이야기를 전한다. 백상예술대상 수상자인 박정민과 큰 자기 유재석의 케미스트리도 재미를 더할 전망이다.\n\r\n열정과 추진력으로 대기업 최초 여성 CEO가 된 윤여순 자기님과의 토크도 이어진다. 40대의 나이에 대기업 부장으로 입사, 4년 만에 임원으로 승진한 윤여순은 자신만의 리더십을 밝힐 예정이다. \n\r\n첫 여성 임원이라는 타이틀의 무게감, 항상 사직서를 품고 다녔던 이유, 유일한 여성 임원으로서 겪은 황당한 에피소드는 물론 2014년 퇴임 후 비즈니스 코칭가로 변신한 근황을 솔직하게 전한다. 특히 윤여정 배우의 동생인 윤여순이 들려주는 \'오스카\' 수상 뒷이야기도 예고돼 기대감을 높인다.\n\r\n세 번의 심장 수술을 이겨내고 의사가 된 신승건 자기님은 불굴의 도전 정신으로 눈길을 끈다. 수술 전날 밤 병실 창문에 비친 의학도서관의 불빛에 이끌려 의사의 꿈을 갖게 되고, 어려움 속에서도 쉬지 않고 매진한 끝에 꿈을 이뤄낸 이야

In [89]:
print(article) # |n -> 줄바꿈



[서울=뉴시스]2일 방송되는 tvN '유 퀴즈 온 더 블럭'. (사진=tvN '유 퀴즈 온 더 블럭' 제공) 2021.06.02. photo@newsis.com[서울=뉴시스] 강진아 기자 = tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동생 윤여순 등이 출연한다.

2일 오후 8시40분에 방송되는 '유 퀴즈 온 더 블럭' 109회에서는 포기를 모르는 끝판왕 자기님들과 '끝날 때까지 끝난 게 아니다' 특집을 선보인다.

배우 박정민, 대기업 첫 여성 임원 윤여순, 의사 신승건, '플랭크' 대가 김영달 자기님이 출연해 끝없이 경주하는 인생 이야기를 나눈다.

먼저 장르 불문 독보적인 연기력으로 충무로 캐스팅 0순위에 오른 배우 박정민 자기님이 '유 퀴즈'를 찾는다. 

오로지 연기에 대한 열정으로 명문대 대신 한예종을 선택한 사연부터 전설로 남은 한예종 면접 비하인드 스토리, 무명시절을 거쳐 지금에 이르기까지 영화보다 더 영화 같은 이야기를 전한다. 백상예술대상 수상자인 박정민과 큰 자기 유재석의 케미스트리도 재미를 더할 전망이다.

열정과 추진력으로 대기업 최초 여성 CEO가 된 윤여순 자기님과의 토크도 이어진다. 40대의 나이에 대기업 부장으로 입사, 4년 만에 임원으로 승진한 윤여순은 자신만의 리더십을 밝힐 예정이다. 

첫 여성 임원이라는 타이틀의 무게감, 항상 사직서를 품고 다녔던 이유, 유일한 여성 임원으로서 겪은 황당한 에피소드는 물론 2014년 퇴임 후 비즈니스 코칭가로 변신한 근황을 솔직하게 전한다. 특히 윤여정 배우의 동생인 윤여순이 들려주는 '오스카' 수상 뒷이야기도 예고돼 기대감을 높인다.

세 번의 심장 수술을 이겨내고 의사가 된 신승건 자기님은 불굴의 도전 정신으로 눈길을 끈다. 수술 전날 밤 병실 창문에 비친 의학도서관의 불빛에 이끌려 의사의 꿈을 갖게 되고, 어려움 속에서도 쉬지 않고 매진한 끝에 꿈을 이뤄낸 이야기를 공개한다. 의사 면허증을 받자마자 제일 먼저 찾아간 예상치 못한 인물에 모두 깜

기사 본문에서 필요없는 부분 삭제

In [100]:
news_content = article.replace('[서울=뉴시스]', '')
news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
news_content = news_content.replace('\n\r\n', ' ')
news_content = news_content.replace('\n\n\n\t', ' ')

In [101]:
print(news_content)



2일 방송되는 tvN '유 퀴즈 온 더 블럭'. (사진=tvN '유 퀴즈 온 더 블럭' 제공) 2021.06.02. photo@newsis.com 강진아 기자 = tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동생 윤여순 등이 출연한다. 2일 오후 8시40분에 방송되는 '유 퀴즈 온 더 블럭' 109회에서는 포기를 모르는 끝판왕 자기님들과 '끝날 때까지 끝난 게 아니다' 특집을 선보인다. 배우 박정민, 대기업 첫 여성 임원 윤여순, 의사 신승건, '플랭크' 대가 김영달 자기님이 출연해 끝없이 경주하는 인생 이야기를 나눈다. 먼저 장르 불문 독보적인 연기력으로 충무로 캐스팅 0순위에 오른 배우 박정민 자기님이 '유 퀴즈'를 찾는다.  오로지 연기에 대한 열정으로 명문대 대신 한예종을 선택한 사연부터 전설로 남은 한예종 면접 비하인드 스토리, 무명시절을 거쳐 지금에 이르기까지 영화보다 더 영화 같은 이야기를 전한다. 백상예술대상 수상자인 박정민과 큰 자기 유재석의 케미스트리도 재미를 더할 전망이다. 열정과 추진력으로 대기업 최초 여성 CEO가 된 윤여순 자기님과의 토크도 이어진다. 40대의 나이에 대기업 부장으로 입사, 4년 만에 임원으로 승진한 윤여순은 자신만의 리더십을 밝힐 예정이다.  첫 여성 임원이라는 타이틀의 무게감, 항상 사직서를 품고 다녔던 이유, 유일한 여성 임원으로서 겪은 황당한 에피소드는 물론 2014년 퇴임 후 비즈니스 코칭가로 변신한 근황을 솔직하게 전한다. 특히 윤여정 배우의 동생인 윤여순이 들려주는 '오스카' 수상 뒷이야기도 예고돼 기대감을 높인다. 세 번의 심장 수술을 이겨내고 의사가 된 신승건 자기님은 불굴의 도전 정신으로 눈길을 끈다. 수술 전날 밤 병실 창문에 비친 의학도서관의 불빛에 이끌려 의사의 꿈을 갖게 되고, 어려움 속에서도 쉬지 않고 매진한 끝에 꿈을 이뤄낸 이야기를 공개한다. 의사 면허증을 받자마자 제일 먼저 찾아간 예상치 못한 인물에 모두 깜짝 놀랐다고 해 궁금증을 자아낸다. 지치지

In [102]:
# 정규식으로 이메일을 찾아 처음부터 '~ 이메일 000 기자 = '까지 삭제
pattern = re.compile(r'@') # 처음 등장하는 @ 위치
email_address = pattern.search(news_content)
print(email_address)

<re.Match object; span=(72, 73), match='@'>


In [103]:
print(email_address.start()) # 이메일이 시작하는 위치

72


In [104]:
news_content[email_address.start():email_address.start() + 21]

'@newsis.com 강진아 기자 = '

In [105]:
# @ ~ newsis.com 000 기자 = -> 21글자 동일
mail_start = email_address.start()
news_content = news_content[mail_start + 21:]

news_content

'tvN \'유 퀴즈 온 더 블럭\'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동생 윤여순 등이 출연한다. 2일 오후 8시40분에 방송되는 \'유 퀴즈 온 더 블럭\' 109회에서는 포기를 모르는 끝판왕 자기님들과 \'끝날 때까지 끝난 게 아니다\' 특집을 선보인다. 배우 박정민, 대기업 첫 여성 임원 윤여순, 의사 신승건, \'플랭크\' 대가 김영달 자기님이 출연해 끝없이 경주하는 인생 이야기를 나눈다. 먼저 장르 불문 독보적인 연기력으로 충무로 캐스팅 0순위에 오른 배우 박정민 자기님이 \'유 퀴즈\'를 찾는다.  오로지 연기에 대한 열정으로 명문대 대신 한예종을 선택한 사연부터 전설로 남은 한예종 면접 비하인드 스토리, 무명시절을 거쳐 지금에 이르기까지 영화보다 더 영화 같은 이야기를 전한다. 백상예술대상 수상자인 박정민과 큰 자기 유재석의 케미스트리도 재미를 더할 전망이다. 열정과 추진력으로 대기업 최초 여성 CEO가 된 윤여순 자기님과의 토크도 이어진다. 40대의 나이에 대기업 부장으로 입사, 4년 만에 임원으로 승진한 윤여순은 자신만의 리더십을 밝힐 예정이다.  첫 여성 임원이라는 타이틀의 무게감, 항상 사직서를 품고 다녔던 이유, 유일한 여성 임원으로서 겪은 황당한 에피소드는 물론 2014년 퇴임 후 비즈니스 코칭가로 변신한 근황을 솔직하게 전한다. 특히 윤여정 배우의 동생인 윤여순이 들려주는 \'오스카\' 수상 뒷이야기도 예고돼 기대감을 높인다. 세 번의 심장 수술을 이겨내고 의사가 된 신승건 자기님은 불굴의 도전 정신으로 눈길을 끈다. 수술 전날 밤 병실 창문에 비친 의학도서관의 불빛에 이끌려 의사의 꿈을 갖게 되고, 어려움 속에서도 쉬지 않고 매진한 끝에 꿈을 이뤄낸 이야기를 공개한다. 의사 면허증을 받자마자 제일 먼저 찾아간 예상치 못한 인물에 모두 깜짝 놀랐다고 해 궁금증을 자아낸다. 지치지 않는 열정으로 인생 3막을 즐기고 있는 김영달 자기님을 알아가는 시간도 마련된다. 86세의 나이에 \'7분 플랭크(Plank) 운동\'으로 하

In [106]:
pattern = re.compile(r'\'')
news_content = pattern.sub('', news_content) # substitute
news_content

'tvN 유 퀴즈 온 더 블럭에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동생 윤여순 등이 출연한다. 2일 오후 8시40분에 방송되는 유 퀴즈 온 더 블럭 109회에서는 포기를 모르는 끝판왕 자기님들과 끝날 때까지 끝난 게 아니다 특집을 선보인다. 배우 박정민, 대기업 첫 여성 임원 윤여순, 의사 신승건, 플랭크 대가 김영달 자기님이 출연해 끝없이 경주하는 인생 이야기를 나눈다. 먼저 장르 불문 독보적인 연기력으로 충무로 캐스팅 0순위에 오른 배우 박정민 자기님이 유 퀴즈를 찾는다.  오로지 연기에 대한 열정으로 명문대 대신 한예종을 선택한 사연부터 전설로 남은 한예종 면접 비하인드 스토리, 무명시절을 거쳐 지금에 이르기까지 영화보다 더 영화 같은 이야기를 전한다. 백상예술대상 수상자인 박정민과 큰 자기 유재석의 케미스트리도 재미를 더할 전망이다. 열정과 추진력으로 대기업 최초 여성 CEO가 된 윤여순 자기님과의 토크도 이어진다. 40대의 나이에 대기업 부장으로 입사, 4년 만에 임원으로 승진한 윤여순은 자신만의 리더십을 밝힐 예정이다.  첫 여성 임원이라는 타이틀의 무게감, 항상 사직서를 품고 다녔던 이유, 유일한 여성 임원으로서 겪은 황당한 에피소드는 물론 2014년 퇴임 후 비즈니스 코칭가로 변신한 근황을 솔직하게 전한다. 특히 윤여정 배우의 동생인 윤여순이 들려주는 오스카 수상 뒷이야기도 예고돼 기대감을 높인다. 세 번의 심장 수술을 이겨내고 의사가 된 신승건 자기님은 불굴의 도전 정신으로 눈길을 끈다. 수술 전날 밤 병실 창문에 비친 의학도서관의 불빛에 이끌려 의사의 꿈을 갖게 되고, 어려움 속에서도 쉬지 않고 매진한 끝에 꿈을 이뤄낸 이야기를 공개한다. 의사 면허증을 받자마자 제일 먼저 찾아간 예상치 못한 인물에 모두 깜짝 놀랐다고 해 궁금증을 자아낸다. 지치지 않는 열정으로 인생 3막을 즐기고 있는 김영달 자기님을 알아가는 시간도 마련된다. 86세의 나이에 7분 플랭크(Plank) 운동으로 하루를 시작하는 자기님은 알파벳도 모르던 학생이 영어

- 발행언론사

In [115]:
article[-26:-1]

'공감언론 뉴시스 akang@newsis.com'

In [118]:
source.find('div', {'class': 'copy'}).get_text()

'Copyright © NEWSIS.COM, 무단 전재 및 재배포 금지'

<br>

### 여러 뉴스 기사 모으기

In [119]:
urls_list

['http://starin.edaily.co.kr/news/newspath.asp?newsid=01151286629084016',
 'https://view.asiae.co.kr/article/2021062109425167742',
 'https://star.ytn.co.kr/_sn/0117_202106110937275511',
 'http://yna.kr/AKR20210607103700005?did=1195m',
 'https://sports.donga.com/article/all/20210618/107500147/1',
 'https://www.news1.kr/articles/?4331691',
 'http://www.gukjenews.com/news/articleView.html?idxno=2243478',
 'http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600',
 'https://www.xportsnews.com/?ac=article_view&entry_id=1435543',
 'https://www.news1.kr/articles/?4326866']

#### <span style='color:orange;'>언론사마다 뉴스 본문이 들어있는 div가 다르기 때문에 뉴시스 언론사만 기사 본문 스크랩할 예정</span>

In [143]:
titles = []
dates = []
articles = []
press_companies = []
urls = []

In [144]:
for url in urls_list:
    if url.startswith('http://www.newsis.com'):
        try:
            content = requests.get(url).content
            source = BeautifulSoup(content, 'html.parser')

            # 기사제목
            title = source.find('h1').get_text()

            # 기사날짜
            date = source.find('div', {'class': 'date'}).get_text()[3:]

            # 기사본문
            article = source.find('div', {'id': 'textBody'}).get_text()
            news_content = article.replace('[서울=뉴시스]', '')
            news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
            news_content = news_content.replace('\n\r\n', ' ')
            news_content = news_content.replace('\n\n\n\t', ' ')
            pattern = re.compile(r'@') # 처음 등장하는 @ 위치
            email_address = pattern.search(news_content)
            mail_start = email_address.start()
            news_content = news_content[mail_start + 21:]

            # 언론사
            press_company = article[-26:-1]
            
            titles.append(title)
            dates.append(date)
            articles.append(news_content)
            press_companies.append(press_company)
            urls.append(url)
            print('Title: {}'.format(url))
            
        except:
            print('*** error occurred at {} ***'.format(url))
    else:
        print('** (뉴시스x) Title: {}'.format(url))

** (뉴시스x) Title: http://starin.edaily.co.kr/news/newspath.asp?newsid=01151286629084016
** (뉴시스x) Title: https://view.asiae.co.kr/article/2021062109425167742
** (뉴시스x) Title: https://star.ytn.co.kr/_sn/0117_202106110937275511
** (뉴시스x) Title: http://yna.kr/AKR20210607103700005?did=1195m
** (뉴시스x) Title: https://sports.donga.com/article/all/20210618/107500147/1
** (뉴시스x) Title: https://www.news1.kr/articles/?4331691
** (뉴시스x) Title: http://www.gukjenews.com/news/articleView.html?idxno=2243478
Title: http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600
** (뉴시스x) Title: https://www.xportsnews.com/?ac=article_view&entry_id=1435543
** (뉴시스x) Title: https://www.news1.kr/articles/?4326866


In [145]:
df_article = pd.DataFrame({'Title': titles, 'Date': dates, 'Article': articles, 'URL': url, 'Press Company': press_companies})
df_article

,Title,Date,Article,URL,Press Company
0,'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연,2021-06-02 13:51:26,tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동...,https://www.news1.kr/articles/?4326866,공감언론 뉴시스 akang@newsis.com


엑셀 파일로 저장

In [147]:
df_article.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

In [149]:
df_article_read = pd.read_excel('result_210621_2328.xlsx', encoding='utf-8')
df_article.head()

,Title,Date,Article,URL,Press Company
0,'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연,2021-06-02 13:51:26,tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동...,https://www.news1.kr/articles/?4326866,공감언론 뉴시스 akang@newsis.com


<br>

### 여러 페이지에 걸쳐 크롤링하기

![page_nums](image/pagenums.png)

메서드화

In [3]:
def scraping_news(url, titles, dates, articles, press_companies, urls):
    if url.startswith('http://www.newsis.com'):
        try:
            content = requests.get(url).content
            source = BeautifulSoup(content, 'html.parser')

            # 기사제목
            title = source.find('h1').get_text()

            # 기사날짜
            date = source.find('div', {'class': 'date'}).get_text()[3:]

            # 기사본문
            article = source.find('div', {'id': 'textBody'}).get_text()
            news_content = article.replace('[서울=뉴시스]', '')
            news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
            news_content = news_content.replace('\n\r\n', ' ')
            news_content = news_content.replace('\n\n\n\t', ' ')
            pattern = re.compile(r'@') # 처음 등장하는 @ 위치
            email_address = pattern.search(news_content)
            mail_start = email_address.start()
            news_content = news_content[mail_start + 21:]

            # 언론사
            press_company = article[-26:-1]

            titles.append(title)
            dates.append(date)
            articles.append(news_content)
            press_companies.append(press_company)
            urls.append(url)
            print('Title: {}'.format(url))

        except:
            print('*** error occurred at {} ***'.format(url))
    else:
        print('*** (뉴시스x) Title: {} ***'.format(url))

In [156]:
query = '배우 박정민'
start = 11 # 2 페이지
url = "https://search.naver.com/search.naver?where=news&query=" + query + '&start=' + str(start)
url

'https://search.naver.com/search.naver?where=news&query=배우 박정민&start=11'

In [158]:
query = '배우 박정민'
start = 21 # 3 페이지
url = "https://search.naver.com/search.naver?where=news&query=" + query + '&start=' + str(start)
url

'https://search.naver.com/search.naver?where=news&query=배우 박정민&start=21'

In [164]:
max_page = 5
starts = []

for i in range(1, max_page * 10 + 1, 10):
    print(i)

1
11
21
31
41


원하는 페이지 수 지정하여 각 기사들의 본문 내용 스크랩핑하기

In [4]:
query = (input('검색할 단어를 입력하세요: '))
max_page = (input('검색할 총 페이지수를 입력하세요: '))

current = 1
last = int(max_page) * 10 + 1
base_url = 'https://search.naver.com/search.naver?where=news&query='


titles = []
dates = []
articles = []
press_companies = []
urls = []
while current <= last:
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current))
    
    url = base_url + query + '&start=' + str(current)
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    
    urls_list = []
    for url in source.find_all('a', {'class': 'news_tit'}):
        urls_list.append(url['href'])
        
    for url in urls_list:
        scraping_news(url, titles, dates, articles, press_companies, urls)
#         if url.startswith('http://www.newsis.com'):
#             try:
#                 content = requests.get(url).content
#                 source = BeautifulSoup(content, 'html.parser')

#                 # 기사제목
#                 title = source.find('h1').get_text()

#                 # 기사날짜
#                 date = source.find('div', {'class': 'date'}).get_text()[3:]

#                 # 기사본문
#                 article = source.find('div', {'id': 'textBody'}).get_text()
#                 news_content = article.replace('[서울=뉴시스]', '')
#                 news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
#                 news_content = news_content.replace('\n\r\n', ' ')
#                 news_content = news_content.replace('\n\n\n\t', ' ')
#                 pattern = re.compile(r'@') # 처음 등장하는 @ 위치
#                 email_address = pattern.search(news_content)
#                 mail_start = email_address.start()
#                 news_content = news_content[mail_start + 21:]

#                 # 언론사
#                 press_company = article[-26:-1]

#                 titles.append(title)
#                 dates.append(date)
#                 articles.append(news_content)
#                 press_companies.append(press_company)
#                 urls.append(url)
#                 print('Title: {}'.format(url))

#             except:
#                 print('*** error occurred at {} ***'.format(url))
#         else:
#             print('*** (뉴시스x) Title: {} ***'.format(url))
    
    time.sleep(5)
    current += 10

검색할 단어를 입력하세요: 배우 박정민
검색할 총 페이지수를 입력하세요: 3

1번째 기사글부터 크롤링을 시작합니다.
*** (뉴시스x) Title: http://sports.chosun.com/news/ntype.htm?id=202106230100182070011597&servicedate=20210622 ***
*** (뉴시스x) Title: http://www.mydaily.co.kr/new_yk/html/read.php?newsid=202106221310432518&ext=na&utm_campaign=naver_news&utm_source=naver&utm_medium=related_news ***
*** (뉴시스x) Title: http://starin.edaily.co.kr/news/newspath.asp?newsid=01151286629084016 ***
*** (뉴시스x) Title: https://view.asiae.co.kr/article/2021062109425167742 ***
*** (뉴시스x) Title: https://star.ytn.co.kr/_sn/0117_202106110937275511 ***
*** (뉴시스x) Title: http://yna.kr/AKR20210607103700005?did=1195m ***
*** (뉴시스x) Title: https://sports.donga.com/article/all/20210618/107500147/1 ***
*** (뉴시스x) Title: https://www.news1.kr/articles/?4331691 ***
Title: http://www.newsis.com/view/?id=NISX20210602_0001462205&cID=10601&pID=10600
*** (뉴시스x) Title: http://www.gukjenews.com/news/articleView.html?idxno=2243478 ***

11번째 기사글부터 크롤링을 시작합니다.
*** (뉴시스x) Title: ht

In [170]:
titles

["'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연",
 '\'유퀴즈\' 박정민, 고려대→한예종 "나는 \'안 될 놈\' 같았다"',
 '박정민 "윤여정 아카데미상 수상...대한민국에 좋은 선물"',
 "'기적' 박정민·이성민·임윤아·이수경, '4色 캐릭터' 예고편 공개",
 '박정민 "\'기적\', 출연 고민했는데 감독이 펭수 굿즈 챙겨줘 믿음"',
 '박정민, 청룡 남우조연상…트랜스젠더 연기 호평',
 '[2021 백상]오정세·염혜란·박정민·김선영, 남녀 조연상 ',
 '故 박지선 향한 박정민의 진심 뭉클..."괜찮냐고 묻지 못해 미안" ']

In [171]:
dates

['2021-06-02 13:51:26',
 '2021-06-03 14:07:02',
 '2021-04-26 16:33:41',
 '2021-05-06 16:18:24',
 '2021-04-26 17:33:57',
 '2021-02-09 22:23:12',
 '2021-05-13 23:09:58',
 '2021-02-10 12:03:03']

데이터프레임 -> 엑셀파일

In [173]:
df_article = pd.DataFrame({'Title':titles, 'Date':dates, 'Article':articles, 'URL':urls, 'Press Company': press_companies})
df_article

,Title,Date,Article,URL,Press Company
0,'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연,2021-06-02 13:51:26,tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동...,http://www.newsis.com/view/?id=NISX20210602_00...,공감언론 뉴시스 akang@newsis.com
1,"'유퀴즈' 박정민, 고려대→한예종 ""나는 '안 될 놈' 같았다""",2021-06-03 14:07:02,금지성유민 인턴 기자 = 배우 박정민이 5년의 무명 시절로 자격지심을 가졌었다고 ...,http://www.newsis.com/view/?id=NISX20210603_00...,감언론 뉴시스 ymsung@newsis.com
2,"박정민 ""윤여정 아카데미상 수상...대한민국에 좋은 선물""",2021-04-26 16:33:41,= 오는 6월 개봉하는 영화 '기적'의 출연진이 배우 윤여정의 아카데미 여우조연상 ...,http://www.newsis.com/view/?id=NISX20210426_00...,론 뉴시스 jmstal01@newsis.com
3,"'기적' 박정민·이성민·임윤아·이수경, '4色 캐릭터' 예고편 공개",2021-05-06 16:18:24,"금지성유민 인턴 기자 = 배우 박정민, 이성민, 임윤아, 이수경의 첫 만남으로 기...",http://www.newsis.com/view/?id=NISX20210506_00...,감언론 뉴시스 ymsung@newsis.com
4,"박정민 ""'기적', 출연 고민했는데 감독이 펭수 굿즈 챙겨줘 믿음""",2021-04-26 17:33:57,금지\r\n 임종명 기자 = 배우 박정민이 영화 '기적'의 출연을 고심하다 이장훈...,http://www.newsis.com/view/?id=NISX20210426_00...,론 뉴시스 jmstal01@newsis.com
5,"박정민, 청룡 남우조연상…트랜스젠더 연기 호평",2021-02-09 22:23:12,= 배우 박정민이 9일 인천 영종도 파라다이스시티에서 열린 제41회 청룡영화상 시상...,http://www.newsis.com/view/?id=NISX20210209_00...,론 뉴시스 jmstal01@newsis.com
6,"[2021 백상]오정세·염혜란·박정민·김선영, 남녀 조연상",2021-05-13 23:09:58,우 오정세·염혜란·박정민·김선영이 '제57회 백상예술대상'에서 조연상을 받았다. ...,http://www.newsis.com/view/?id=NISX20210513_00...,뉴시스 realpaper7@newsis.com
7,"故 박지선 향한 박정민의 진심 뭉클...""괜찮냐고 묻지 못해 미안""",2021-02-10 12:03:03,"""만약에 제가 이 마이크 앞에서 딱 한 사람에게 진심으로 감사의 말을 할 수 있다면...",http://www.newsis.com/view/?id=NISX20210210_00...,스 lovelypsyche@newsis.com


엑셀 파일로 저장

In [5]:
datetime.now()

datetime.datetime(2021, 6, 23, 0, 17, 11, 785551)

In [174]:
df_article.to_excel('resultAll_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

In [175]:
df_read = pd.read_excel('resultAll_210621_2356.xlsx', encoding='utf-8')
df_read

,Title,Date,Article,URL,Press Company
0,'유 퀴즈' 배우 박정민→윤여정 동생 윤여순 출연,2021-06-02 13:51:26,tvN '유 퀴즈 온 더 블럭'에 배우 박정민과 대기업 첫 여성 임원인 윤여정의 동...,http://www.newsis.com/view/?id=NISX20210602_00...,공감언론 뉴시스 akang@newsis.com
1,"'유퀴즈' 박정민, 고려대→한예종 ""나는 '안 될 놈' 같았다""",2021-06-03 14:07:02,금지성유민 인턴 기자 = 배우 박정민이 5년의 무명 시절로 자격지심을 가졌었다고 ...,http://www.newsis.com/view/?id=NISX20210603_00...,감언론 뉴시스 ymsung@newsis.com
2,"박정민 ""윤여정 아카데미상 수상...대한민국에 좋은 선물""",2021-04-26 16:33:41,0,http://www.newsis.com/view/?id=NISX20210426_00...,론 뉴시스 jmstal01@newsis.com
3,"'기적' 박정민·이성민·임윤아·이수경, '4色 캐릭터' 예고편 공개",2021-05-06 16:18:24,"금지성유민 인턴 기자 = 배우 박정민, 이성민, 임윤아, 이수경의 첫 만남으로 기...",http://www.newsis.com/view/?id=NISX20210506_00...,감언론 뉴시스 ymsung@newsis.com
4,"박정민 ""'기적', 출연 고민했는데 감독이 펭수 굿즈 챙겨줘 믿음""",2021-04-26 17:33:57,금지\r\n 임종명 기자 = 배우 박정민이 영화 '기적'의 출연을 고심하다 이장훈...,http://www.newsis.com/view/?id=NISX20210426_00...,론 뉴시스 jmstal01@newsis.com
5,"박정민, 청룡 남우조연상…트랜스젠더 연기 호평",2021-02-09 22:23:12,0,http://www.newsis.com/view/?id=NISX20210209_00...,론 뉴시스 jmstal01@newsis.com
6,"[2021 백상]오정세·염혜란·박정민·김선영, 남녀 조연상",2021-05-13 23:09:58,우 오정세·염혜란·박정민·김선영이 '제57회 백상예술대상'에서 조연상을 받았다. ...,http://www.newsis.com/view/?id=NISX20210513_00...,뉴시스 realpaper7@newsis.com
7,"故 박지선 향한 박정민의 진심 뭉클...""괜찮냐고 묻지 못해 미안""",2021-02-10 12:03:03,"""만약에 제가 이 마이크 앞에서 딱 한 사람에게 진심으로 감사의 말을 할 수 있다면...",http://www.newsis.com/view/?id=NISX20210210_00...,스 lovelypsyche@newsis.com


<br>

### 날짜 지정하여 검색

In [1]:
# 메서드
def scraping_news(url, titles, dates, articles, press_companies, urls):
    if url.startswith('http://www.newsis.com'):
        try:
            content = requests.get(url).content
            source = BeautifulSoup(content, 'html.parser')

            # 기사제목
            title = source.find('h1').get_text()

            # 기사날짜
            date = source.find('div', {'class': 'date'}).get_text()[3:]

            # 기사본문
            article = source.find('div', {'id': 'textBody'}).get_text()
            news_content = article.replace('[서울=뉴시스]', '')
            news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
            news_content = news_content.replace('\n\r\n', ' ')
            news_content = news_content.replace('\n\n\n\t', ' ')
            pattern = re.compile(r'@') # 처음 등장하는 @ 위치
            email_address = pattern.search(news_content)
            mail_start = email_address.start()
            news_content = news_content[mail_start + 21:]

            # 언론사
            press_company = article[-26:-1]

            titles.append(title)
            dates.append(date)
            articles.append(news_content)
            press_companies.append(press_company)
            urls.append(url)
            print('Title: {}'.format(url))

        except:
            print('*** error occurred at {} ***'.format(url))
    else:
        print('*** (뉴시스x) Title: {} ***'.format(url))

In [189]:
query = (input('검색할 단어를 입력하세요: '))

start_date = '2021-06-22'.replace('.','')
end_date='2021-06-20'.replace('.','')

current = 1
last = int(max_page) * 10 + 1
base_url = 'https://search.naver.com/search.naver?where=news&query='


titles = []
dates = []
articles = []
press_companies = []
urls = []
# while current <= last:
print('크롤링을 시작합니다.')

url = base_url + query + '&nso=so%3Ar%2Cp%3Afrom' + start_date + 'to' + end_date +  '%2Ca%3A&start=1'
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

urls_list = []
for url in source.find_all('a', {'class': 'news_tit'}):
    urls_list.append(url['href'])

for url in urls_list:
    scraping_news(url, titles, dates, articles, press_companies, urls)
#     if url.startswith('http://www.newsis.com'):
#         try:
#             content = requests.get(url).content
#             source = BeautifulSoup(content, 'html.parser')

#             # 기사제목
#             title = source.find('h1').get_text()

#             # 기사날짜
#             date = source.find('div', {'class': 'date'}).get_text()[3:]

#             # 기사본문
#             article = source.find('div', {'id': 'textBody'}).get_text()
#             news_content = article.replace('[서울=뉴시스]', '')
#             news_content = news_content.replace('◎공감언론 뉴시스 akang@newsis.com', '')
#             news_content = news_content.replace('\n\r\n', ' ')
#             news_content = news_content.replace('\n\n\n\t', ' ')
#             pattern = re.compile(r'@') # 처음 등장하는 @ 위치
#             email_address = pattern.search(news_content)
#             mail_start = email_address.start()
#             news_content = news_content[mail_start + 21:]

#             # 언론사
#             press_company = article[-26:-1]

#             titles.append(title)
#             dates.append(date)
#             articles.append(news_content)
#             press_companies.append(press_company)
#             urls.append(url)
#             print('Title: {}'.format(url))

#         except:
#             print('*** error occurred at {} ***'.format(url))
#     else:
#         print('*** (뉴시스x) Title: {} ***'.format(url))

검색할 단어를 입력하세요: 10cm
크롤링을 시작합니다.
*** (뉴시스x) Title: http://news.mk.co.kr/newsRead.php?no=540986&year=2021 ***
*** (뉴시스x) Title: http://www.mydaily.co.kr/new_yk/html/read.php?newsid=202106141656466696&ext=na&utm_campaign=naver_news&utm_source=naver&utm_medium=related_news ***
*** (뉴시스x) Title: https://www.xportsnews.com/?ac=article_view&entry_id=1435237 ***
*** (뉴시스x) Title: http://www.inews24.com/view/1372945 ***
*** (뉴시스x) Title: https://www.news1.kr/articles/?4305233 ***
Title: http://www.newsis.com/view/?id=NISX20210507_0001433069&cID=10601&pID=10600
*** (뉴시스x) Title: http://sports.khan.co.kr/news/sk_index.html?art_id=202105141304003&sec_id=540301&pt=nv ***
*** (뉴시스x) Title: https://biz.chosun.com/site/data/html_dir/2021/03/05/2021030501092.html?utm_source=naver&utm_medium=original&utm_campaign=biz ***
*** (뉴시스x) Title: http://news.mt.co.kr/mtview.php?no=2021020313454174795 ***
*** (뉴시스x) Title: http://www.fnnews.com/news/202106021434103152 ***


In [190]:
def make_dateframe(titles, dates, articles, urls, presscompanies):
    return pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':urls, 
                           'PressCompany':presscompanies})

In [193]:
df_10cm = make_dateframe(titles, dates, articles, urls, press_companies)
df_10cm

,Title,Date,Article,URL,PressCompany
0,"10CM, 신곡 '서울의 잠 못 이루는 밤' 13일 발매",2021-05-07 11:43:04,금지\r\n 남정현 기자 = 가수 10CM(십센치)가 신곡 '서울의 잠 못 이루는...,http://www.newsis.com/view/?id=NISX20210507_00...,감언론 뉴시스 nam_jh@newsis.com


<br>

### 정렬 순서 지정하여 크롤링하기

In [ ]:
query = (input('검색할 단어를 입력하세요: '))
max_page = (input('검색할 총 페이지수를 입력하세요: '))
sort_type = (input('정렬 순서를 입력하세요(0:관련도순, 1:최신순, 2:오래된순: '))

current = 1
last = int(max_page) * 10 + 1
base_url = 'https://search.naver.com/search.naver?where=news&query='


titles = []
dates = []
articles = []
press_companies = []
urls = []

while current <= last:
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current))
    
    url = base_url + query + "&sort=" + str(sort_type) + "&start=" + str(current)
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    
    urls_list = []
    for item in source.find_all('a', {'class': 'news_tit'}):
        urls_list.append(item['href'])
    
    for url in urls_list:
        scraping_news(url, titles, dates, articles, press_companies, urls)
    
    time.sleep(5)
    current += 10

In [ ]:
df = make_dateframe(titles, dates, articles, urls, press_companies)
df